In [ ]:
#使用帕克螺旋和PFSS实现从卫星观测到的太阳风回溯到光球层表面足点

import astropy.constants as const
import astropy.units as u
from datetime import datetime,timedelta
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
import sunkit_magex.pfss
from sunkit_magex.pfss import tracing
import os
import numpy as np

#变量说明：全部以‘列表’格式输入（adapt_data_path 除外），其中Uly_date列表中存储 datetiem 格式的元素；Uly_heliocentricDistance列表中存储以 AU 为单位的浮点数；Uly_plasmaFlowSpeed列表中存储以 km/s 为单位的浮点数；
#        Uly_heliographicLatitude和Uly_heliographicLongitude列表中存储以 deg 为单位的浮点数；adapt_data_path是存放磁图数据的文件夹，请确保研究时间范围内每天至少有一张磁图，可以在（https://gong.nso.edu/adapt/maps/gong/）这里下载磁图。

def caculate_footpoints(Uly_date,Uly_heliocentricDistance,Uly_plasmaFlowSpeed,Uly_heliographicLatitude,Uly_heliographicLongitude,adapt_data_path):
    t_travel = ((Uly_heliocentricDistance*const.au-2.5*const.R_sun)/((Uly_plasmaFlowSpeed*10**3)*(u.m/u.s))).value # 传播时间，单位为s；2.5 为太阳源表面半径
    latitude_trans = (np.where(Uly_heliographicLatitude<0,90+Uly_heliographicLatitude,90-Uly_heliographicLatitude))*np.pi/180
    longitude_ss = (Uly_heliographicLongitude+(t_travel/86400)*14.1844*np.sin(latitude_trans)) % 360 # 14.1844为太阳自转率/天
    r_ss_list = np.full(len(longitude_ss),2.5)
    HGI_coord = SkyCoord(longitude_ss*u.deg,# 这个变量就是帕克螺旋后，在源表面的足点信息
                         Uly_heliographicLatitude*u.deg,
                         r_ss_list*const.R_sun,
                         obstime=Uly_date,
                         frame=frames.HeliocentricInertial)
    seeds_Carr = HGI_coord.transform_to(frames.HeliographicCarrington(observer="earth"))
    r = 2.5 * const.R_sun
    lat = np.radians(seeds_Carr.lat.value)*u.rad# 在这里多此一举是因为后面要用到与磁图时间一致的足点但是 skycoord 格式的 seeds_Carr 的时间不可更改
    lon = np.radians(seeds_Carr.lon.value)*u.rad
    
    T_to_Ulysses = ((Uly_heliocentricDistance*const.au-1*const.R_sun)/((Uly_plasmaFlowSpeed*10**3)*(u.m/u.s))).value # 传播时间，单位为s；1 为太阳半径
    adapt_days = {}#这个变量保存了原始Ulysses数据与按足点时间整理的变量之间的关系
    for ii in range(len(Uly_date)):#这段循环中的ii代表索引
        footpoints_date = Uly_date[ii]-timedelta(seconds=T_to_Ulysses[ii])
        if str(footpoints_date)[0:10] in adapt_days:#将数据在太阳表面的时间按照日期分类，如果想要更高精度可以下载更多的磁图，然后把这里改成每小时/分钟都可以，但是我觉得没必要
            adapt_days[str(footpoints_date)[:10]][0].append(footpoints_date)
            adapt_days[str(footpoints_date)[:10]][1].append(ii)
        else:
            adapt_days[str(footpoints_date)[:10]] = [[footpoints_date],[ii]]
            
    # adapt_files = os.listdir(adapt_data_path)
    field_lines_record = []
    for ii in adapt_days:#这段循环中的ii代表日期
        # for file in adapt_files: #这里注释掉是因为我之前写的代码是每天一个磁图，而马哥只需要一个磁图，所以这里注释掉了。你用的时候看需求，如果每天一个磁图就把让我改下就好了
        #     if not os.path.isdir(file):
        #         filetime = str(datetime.strptime(file[20:28],'%Y%m%d'))[:10]
        #         if filetime == ii:
        #             adapt_data = sunkit_magex.pfss.utils.load_adapt(adapt_data_path+file)
        #             print('选取的磁图文件为：',file)
        #             break
        adapt_data = sunkit_magex.pfss.utils.load_adapt(adapt_data_path)
        adapt_map = sunkit_magex.pfss.utils.car_to_cea(adapt_data[0],method='interp')
        nrho = 100 # the number of rho grid points
        rss = 2.5 # 势场源头表面半径
        pfss_in = sunkit_magex.pfss.Input(adapt_map,nrho,rss)
        pfss_out = sunkit_magex.pfss.pfss(pfss_in)
        tracer = tracing.FortranTracer()
        seeds_parkered = SkyCoord(lon[adapt_days[ii][1]],#这里实现了生成一个需要的坐标的时间（有点傻，但我没更好的办法）
                                  lat[adapt_days[ii][1]],
                                  r, frame=pfss_out.coordinate_frame)
        field_lines = tracer.trace(seeds_parkered,pfss_out)
        field_lines_record.append(field_lines)
    return field_lines_record,pfss_out,adapt_days,adapt_data

#一定要注意 field_lines 这个变量的使用，其中每一根磁力线按照磁场极性的不同排列方式相反（磁场为正极性时，第一个点是光球层足点；磁场为负极性时，最后一个点才是光球层足点）。
#关于field_lines的具体使用方法可以查阅 Sunpy 的文档（https://docs.sunpy.org/projects/sunkit-magex/en/stable/generated/gallery/index.html）
#此外，field_lines得到的所有点的时间都是对应磁图的时间，如果要将足点绘制在除对应磁图之外的图像上时（比如要 overlap 到 EUV 图像上）需要将每跟磁力线上每一个点的时间更改为正确的足点的时间就是上面代码中 adapt_days[ii][0]内存储的时间，如何修改 skycoord 的时间代码中也有体现，照葫芦画瓢就好了。

In [ ]:
#用于测试上述代码，不需要参考，反正按照说明把数据塞进函数里就好了
import Unkonw
import numpy as np
import matplotlib.pyplot as plt
import pickle
adapt_data_path = '/mnt/hc320/shares/zhangwn/input/test/adapt40311_03k012_202111181000_i00023600n1.fts'
OMIN_data_path = '/mnt/hc320/shares/zhangwn/input/test/OMIN_data.pkl'
pickle_file = '/mnt/hc320/shares/zhangwn/output/OMIN_LINES.pkl'
data = Unkonw.read_insuit_pkl(OMIN_data_path)

lines,pfss_out,adapt_days,adapt_data = caculate_footpoints(data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['DATE'],
                            np.ones(len(data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['DATE'])),
                            data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['V'],
                            data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['heliographicLatitude'],
                            data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['heliographicLongitude'],
                            adapt_data_path)#就是这一行用了上一个代码块定义的函数，这里往下就是具体实现其他功能的了。马哥需要往磁图点足点，你看你要啥自己加或者我加上都行。

with open(pickle_file,'wb') as file:
    pickle.dump(lines,file)

Ulyssed_position = SkyCoord(data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['heliographicLongitude']*u.deg,
                            data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['heliographicLatitude']*u.deg,
                            np.ones(len(data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['DATE']))*const.au,
                            obstime=data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['DATE'],
                            frame=frames.HeliocentricInertial)
Ulyssed_position = Ulyssed_position.transform_to(frames.HeliographicCarrington(observer="earth"))#经网站验证计算的无误！！卡林顿坐标没毛病
lat_U = np.radians(Ulyssed_position.lat.value)*u.rad
lon_U = np.radians(Ulyssed_position.lon.value)*u.rad


Ulyssed_position_trans = SkyCoord(lon_U,lat_U,np.ones(len(data['OMNI_COHO1HR_MERGED_MAG_PLASMA']['DATE']))*const.au,frame=pfss_out.coordinate_frame)
right_footpoints_Carr = []
right_footpoints_HSH = []

#绘制磁图
fig = plt.figure(figsize=(20,10))
ax = plt.subplot(projection=adapt_data[0])
adapt_data[0].plot(cmap='RdBu_r',vmin=-50,vmax=50)#RdBu
cba = plt.colorbar()

    #绘制箭头,这个磁力线的数据十分诡异，不同磁场极性绘制的方向还不同
record_maxi = 0
for field_lines_HGI in lines:
    for i in range(len(field_lines_HGI)):#绘制线段，线段的起点是Ulysses位置，终点是磁力线足点
        if len(field_lines_HGI[i].coords) == 0:
            continue
        if field_lines_HGI[i].polarity==1:
            end = field_lines_HGI[i].coords[0]  # 终点
            start = Ulyssed_position_trans[record_maxi+i]  # 起点
            #ax.plot([start.lon.deg,end.lon.deg],[start.lat.deg+89,end.lat.deg+89],'r-',linewidth=.5)
            new_combine_coord = SkyCoord([start.lon,end.lon],[start.lat,end.lat],[start.radius,end.radius],frame=pfss_out.coordinate_frame)
            ax.plot_coord(new_combine_coord,color='red',linewidth=0.5)
        else:
            end = field_lines_HGI[i].coords[-1]  # 终点
            start = Ulyssed_position_trans[record_maxi+i]  # 起点
            #ax.plot([start.lon.deg,end.lon.deg],[start.lat.deg+89,end.lat.deg+89],'b-',linewidth=.5)
            new_combine_coord = SkyCoord([start.lon,end.lon],[start.lat,end.lat],[start.radius,end.radius],frame=pfss_out.coordinate_frame)
            ax.plot_coord(new_combine_coord,color='blue',linewidth=0.5)
    record_maxi = record_maxi + i + 1
    for i in range(len(field_lines_HGI)):#给线段的终点也就是磁力线的足点加绿点
        if len(field_lines_HGI[i].coords) == 0:
            continue
        if field_lines_HGI[i].polarity==1:
            ax.plot_coord(field_lines_HGI[i].coords[0],'o',color='green',markersize=2)
        else:
            ax.plot_coord(field_lines_HGI[i].coords[-1],'o',color='green',markersize=2)
ax.set_title('Photospheric field and traced field lines',fontsize=20)
ax.set_xlabel('Carrington Longitude',fontsize=15)
ax.set_ylabel('Latitude',fontsize=15)
ax.tick_params(axis='x',labelsize=15)
ax.tick_params(axis='y',labelsize=15)
cba.ax.tick_params(labelsize=15)

fig.savefig(f'/mnt/hc320/shares/zhangwn/output/test1.png')
plt.clf()
plt.close()

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following metadata is missing: crln_obs,dsun_obs,crlt_obs
 [sunpy.map.mapbase]


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following metadata is missing: crln_obs,dsun_obs,crlt_obs
 [sunpy.map.mapbase]


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


...

For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following metadata is missing: crln_obs,dsun_obs,crlt_obs
 [sunpy.map.mapbase]


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following metadata is missing: crln_obs,dsun_obs,crlt_obs
 [sunpy.map.mapbase]


INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following metadata is missing: crln_obs,dsun_obs,crlt_obs
 [sunpy.map.mapbase]
